In [1]:
# @title Cell 1｜聯邦學習環境設置（完整修正版）
import os
import subprocess
import sys
import warnings
warnings.filterwarnings("ignore")

# =========================================================
# ❶ 徹底清理並重新安裝相容套件
# =========================================================
def install_compatible_federated_environment():
    """徹底重新安裝相容的聯邦學習環境"""
    print("🚀 徹底重新安裝聯邦學習環境...")

    # 步驟1: 徹底卸載相關套件
    packages_to_remove = [
        "tensorflow-federated",
        "tensorflow-privacy",
        "tensorflow-estimator",
        "tensorflow-model-optimization",
        "tensorflow",
        "tf-keras",
        "numpy",
        "dp-accounting"  # 也要移除這個
    ]

    for pkg in packages_to_remove:
        print(f"🗑️ 徹底移除 {pkg}...")
        subprocess.run([
            sys.executable, "-m", "pip", "uninstall", "-y", pkg
        ], capture_output=True, check=False)

    # 步驟2: 清理 pip 快取
    subprocess.run([sys.executable, "-m", "pip", "cache", "purge"],
                   capture_output=True, check=False)

    # 步驟3: 先安裝相容的 NumPy 版本
    print("📦 安裝相容的 NumPy 版本...")
    numpy_install = subprocess.run([
        sys.executable, "-m", "pip", "install", "--no-cache-dir",
        "numpy<2.0"
    ], capture_output=True, text=True)

    if numpy_install.returncode == 0:
        print("✅ NumPy<2.0 安裝成功")
    else:
        print(f"❌ NumPy 安裝失敗: {numpy_install.stderr}")

    # 步驟4: 安裝 dp-accounting（TensorFlow Privacy 的依賴）
    print("📦 安裝 dp-accounting...")
    dp_install = subprocess.run([
        sys.executable, "-m", "pip", "install", "--no-cache-dir",
        "dp-accounting==0.4.3"  # 指定相容版本
    ], capture_output=True, text=True)

    if dp_install.returncode == 0:
        print("✅ dp-accounting 安裝成功")
    else:
        print(f"❌ dp-accounting 安裝失敗: {dp_install.stderr}")

    # 步驟5: 安裝與當前環境相容的版本組合
    print("📦 安裝相容的套件組合...")

    # 使用與 Colab 當前環境相容的版本
    compatible_installs = [
        "tensorflow==2.15.0",
        "tensorflow-estimator==2.15.0",
        "tensorflow-privacy==0.9.0",
        "tensorflow-federated==0.73.0"
    ]

    for install_cmd in compatible_installs:
        print(f"📦 安裝 {install_cmd}")
        result = subprocess.run([
            sys.executable, "-m", "pip", "install", "--no-cache-dir",
            install_cmd
        ], capture_output=True, text=True)

        if result.returncode == 0:
            print(f"✅ {install_cmd} 安裝成功")
        else:
            print(f"❌ {install_cmd} 安裝失敗")
            print(f"   錯誤: {result.stderr}")

    # 步驟6: 安裝其他必要的依賴
    print("📦 安裝其他必要依賴...")
    other_deps = [
        "protobuf>=3.20,<4",
        "absl-py>=1.0.0",
        "attrs>=21.4.0",
        "cachetools>=5.2",
        "dm-tree>=0.1.8",
        "grpcio>=1.48.2",
        "jax>=0.4.1",
        "jaxlib>=0.4.1",
        "portpicker>=1.5.2",
        "semantic-version>=2.10",
        "sortedcontainers>=2.4.0",
        "tqdm>=4.64.1",
        "typing-extensions>=4.5.0",
        "tensorflow-model-optimization>=0.7.3",
        "tensorflow-compression>=2.13",
        "scipy>=1.9.0",
        "scikit-learn>=1.0.0"
    ]

    for dep in other_deps:
        subprocess.run([
            sys.executable, "-m", "pip", "install", "--no-cache-dir", dep
        ], capture_output=True, check=False)

# 執行安裝
install_compatible_federated_environment()

print("\n" + "="*60)
print("⚠️  重要提醒：建議重新啟動 Runtime！")
print("   Runtime > Restart session")
print("   然後重新執行此 Cell 進行驗證")
print("="*60)

# =========================================================
# ❷ 驗證安裝結果
# =========================================================
print("\n🔍 驗證套件安裝...")

# 測試 dp_accounting
try:
    import dp_accounting
    print("✅ dp_accounting 載入成功")
except Exception as e:
    print(f"❌ dp_accounting 載入失敗: {e}")

# 基礎模組
try:
    import json
    import numpy as np
    print(f"✅ NumPy {np.__version__} 載入成功")
    import pandas as pd
    from collections import OrderedDict
    from typing import List, Tuple, Dict, Any, Optional
    print("✅ 基礎模組導入成功")
except Exception as e:
    print(f"❌ 基礎模組導入失敗: {e}")

# TensorFlow 驗證
try:
    import tensorflow as tf
    print(f"✅ TensorFlow {tf.__version__} 載入成功")

    # 設定 TensorFlow 配置
    tf.get_logger().setLevel('ERROR')

    # GPU 設定
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU 已設定: {len(gpus)} 個 GPU 可用")
    else:
        print("📱 使用 CPU 模式")

except Exception as e:
    print(f"❌ TensorFlow 載入失敗: {e}")

# TensorFlow Privacy 驗證
try:
    import tensorflow_privacy as tfp
    print(f"✅ TensorFlow Privacy {tfp.__version__} 載入成功")
except Exception as e:
    print(f"❌ TensorFlow Privacy 載入失敗: {e}")

# TensorFlow Federated 驗證
tff_loaded = False
try:
    import tensorflow_federated as tff
    print(f"✅ TensorFlow Federated {tff.__version__} 載入成功")
    tff_loaded = True
except Exception as e:
    print(f"❌ TensorFlow Federated 載入失敗: {e}")
    print("💡 將使用自實現的聯邦學習核心")

# =========================================================
# ❸ 聯邦學習配置
# =========================================================
FEDERATED_CONFIG = {
    # 基本參數
    "num_clients": 7,
    "base_stations": [1, 2, 3, 4, 5, 6, 7],
    "rounds": 30,
    "local_epochs": 3,
    "batch_size": 512,
    "learning_rate": 0.001,

    # 進階功能設定
    "dp_enabled": False,  # 可以設定為 True 如果 TF Privacy 載入成功
    "secure_aggregation": False,
    "compression_enabled": False,
    "personalization": False,

    # 客戶端選擇
    "clients_per_round": 5,
    "min_available_clients": 3,

    # 差分隱私參數
    "dp_noise_multiplier": 0.1,
    "dp_l2_norm_clip": 1.0
}

print(f"\n🏗️ 聯邦學習環境配置完成")
print(f"📊 配置: {FEDERATED_CONFIG['num_clients']} 客戶端, {FEDERATED_CONFIG['rounds']} 輪次")
print(f"🔒 隱私保護: {'啟用' if FEDERATED_CONFIG['dp_enabled'] else '關閉'}")
print(f"🌐 TensorFlow Federated: {'可用' if tff_loaded else '使用自實現版本'}")

# =========================================================
# ❹ 自實現聯邦學習核心（備選方案）
# =========================================================
class SimpleFederatedLearning:
    """簡化的聯邦學習實現，避免複雜的套件相依性"""

    def __init__(self, config):
        self.config = config
        self.global_model_weights = None
        self.training_history = {
            "rounds": [],
            "avg_loss": [],
            "client_losses": [],
            "train_loss": [],
            "test_loss": []
        }

    def create_model(self, input_dim):
        """創建標準神經網路模型"""
        try:
            import tensorflow as tf

            model = tf.keras.Sequential([
                tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.3),

                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),

                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dropout(0.1),

                tf.keras.layers.Dense(1, activation='sigmoid')
            ])

            model.compile(
                optimizer=tf.keras.optimizers.Adam(self.config["learning_rate"]),
                loss='mse',
                metrics=['mae']
            )

            return model
        except Exception as e:
            print(f"❌ 模型創建失敗: {e}")
            return None

    def federated_averaging(self, client_weights_list, client_sizes):
        """實現聯邦平均算法"""
        if not client_weights_list:
            return None

        # 計算加權平均
        total_size = sum(client_sizes)
        averaged_weights = []

        for layer_idx in range(len(client_weights_list[0])):
            weighted_sum = None

            for client_weights, size in zip(client_weights_list, client_sizes):
                layer_weight = client_weights[layer_idx] * (size / total_size)

                if weighted_sum is None:
                    weighted_sum = layer_weight
                else:
                    weighted_sum += layer_weight

            averaged_weights.append(weighted_sum)

        return averaged_weights

    def train_client(self, client_id, X_train, y_train, X_val, y_val):
        """訓練單一客戶端"""
        # 創建本地模型
        model = self.create_model(X_train.shape[1])

        if model is None:
            return None, 0, {}

        # 如果有全域模型權重，則載入
        if self.global_model_weights is not None:
            model.set_weights(self.global_model_weights)

        # 本地訓練
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=self.config["local_epochs"],
            batch_size=self.config["batch_size"],
            verbose=0
        )

        return model.get_weights(), len(X_train), history.history

print("✅ 簡化聯邦學習類別定義完成")

# 創建聯邦學習系統
federated_system = SimpleFederatedLearning(FEDERATED_CONFIG)

# =========================================================
# ❺ 如果 TFF 載入成功，創建 TFF 版本
# =========================================================
if tff_loaded:
    print("\n🎯 TensorFlow Federated 載入成功，準備 TFF 功能...")

    def create_tff_model():
        """創建 TFF 相容的模型"""
        return SimpleFederatedLearning(FEDERATED_CONFIG).create_model

    print("✅ TFF 模型工廠函數已準備")
else:
    print("\n📌 將使用自實現的聯邦學習系統")

print("\n🎉 聯邦學習環境準備完成！")
print("📋 下一步：執行 Cell 2 進行資料準備")

# 最終環境檢查
print("\n" + "="*60)
print("📊 最終環境狀態檢查：")
modules_status = {
    "NumPy": False,
    "TensorFlow": False,
    "TensorFlow Federated": tff_loaded,
    "TensorFlow Privacy": False,
    "dp_accounting": False
}

try:
    import numpy
    modules_status["NumPy"] = True
except:
    pass

try:
    import tensorflow
    modules_status["TensorFlow"] = True
except:
    pass

try:
    import tensorflow_privacy
    modules_status["TensorFlow Privacy"] = True
except:
    pass

try:
    import dp_accounting
    modules_status["dp_accounting"] = True
except:
    pass

for module, status in modules_status.items():
    print(f"  {module}: {'✅ 可用' if status else '❌ 不可用'}")

if all(modules_status.values()):
    print("\n🚀 所有模組載入成功！可以使用完整功能")
else:
    print("\n⚠️  部分模組未載入，但可以使用基本功能")

print("="*60)

🚀 徹底重新安裝聯邦學習環境...
🗑️ 徹底移除 tensorflow-federated...
🗑️ 徹底移除 tensorflow-privacy...
🗑️ 徹底移除 tensorflow-estimator...
🗑️ 徹底移除 tensorflow-model-optimization...
🗑️ 徹底移除 tensorflow...
🗑️ 徹底移除 tf-keras...
🗑️ 徹底移除 numpy...
🗑️ 徹底移除 dp-accounting...
📦 安裝相容的 NumPy 版本...
✅ NumPy<2.0 安裝成功
📦 安裝 dp-accounting...
✅ dp-accounting 安裝成功
📦 安裝相容的套件組合...
📦 安裝 tensorflow==2.15.0
✅ tensorflow==2.15.0 安裝成功
📦 安裝 tensorflow-estimator==2.15.0
✅ tensorflow-estimator==2.15.0 安裝成功
📦 安裝 tensorflow-privacy==0.9.0
✅ tensorflow-privacy==0.9.0 安裝成功
📦 安裝 tensorflow-federated==0.73.0
✅ tensorflow-federated==0.73.0 安裝成功
📦 安裝其他必要依賴...

⚠️  重要提醒：建議重新啟動 Runtime！
   Runtime > Restart session
   然後重新執行此 Cell 進行驗證

🔍 驗證套件安裝...
✅ dp_accounting 載入成功
✅ NumPy 1.25.2 載入成功
✅ 基礎模組導入成功


ERROR:jax._src.xla_bridge:Jax plugin configuration error: Plugin module %s could not be loaded
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/xla_bridge.py", line 428, in discover_pjrt_plugins
    plugin_module = importlib.import_module(plugin_module_name)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_r

✅ TensorFlow 2.14.1 載入成功
📱 使用 CPU 模式
✅ TensorFlow Privacy 0.9.0 載入成功
✅ TensorFlow Federated 0.73.0 載入成功

🏗️ 聯邦學習環境配置完成
📊 配置: 7 客戶端, 30 輪次
🔒 隱私保護: 關閉
🌐 TensorFlow Federated: 可用
✅ 簡化聯邦學習類別定義完成

🎯 TensorFlow Federated 載入成功，準備 TFF 功能...
✅ TFF 模型工廠函數已準備

🎉 聯邦學習環境準備完成！
📋 下一步：執行 Cell 2 進行資料準備

📊 最終環境狀態檢查：
  NumPy: ✅ 可用
  TensorFlow: ✅ 可用
  TensorFlow Federated: ✅ 可用
  TensorFlow Privacy: ✅ 可用
  dp_accounting: ✅ 可用

🚀 所有模組載入成功！可以使用完整功能


In [1]:
# @title Cell 1｜聯邦學習環境設置（修正版）
import os
import subprocess
import sys
import warnings
warnings.filterwarnings("ignore")

# =========================================================
# ❶ 徹底清理並重新安裝相容套件
# =========================================================
def install_compatible_federated_environment():
    """徹底重新安裝相容的聯邦學習環境"""
    print("🚀 徹底重新安裝聯邦學習環境...")

    # 步驟1: 徹底卸載相關套件
    packages_to_remove = [
        "tensorflow-federated",
        "tensorflow-privacy",
        "tensorflow-estimator",
        "tensorflow-model-optimization",
        "tensorflow",
        "tf-keras",
        "numpy"  # 也要重新安裝 NumPy
    ]

    for pkg in packages_to_remove:
        print(f"🗑️ 徹底移除 {pkg}...")
        subprocess.run([
            sys.executable, "-m", "pip", "uninstall", "-y", pkg
        ], capture_output=True, check=False)

    # 步驟2: 清理 pip 快取
    subprocess.run([sys.executable, "-m", "pip", "cache", "purge"],
                   capture_output=True, check=False)

    # 步驟3: 先安裝相容的 NumPy 版本
    print("📦 安裝相容的 NumPy 版本...")
    numpy_install = subprocess.run([
        sys.executable, "-m", "pip", "install", "--no-cache-dir",
        "numpy<2.0"  # 安裝 NumPy 1.x 版本
    ], capture_output=True, text=True)

    if numpy_install.returncode == 0:
        print("✅ NumPy<2.0 安裝成功")
    else:
        print(f"❌ NumPy 安裝失敗: {numpy_install.stderr}")

    # 步驟4: 安裝與當前環境相容的版本組合
    print("📦 安裝相容的套件組合...")

    # 使用與 Colab 當前環境相容的版本
    compatible_installs = [
        "tensorflow==2.15.0",  # 使用較新但穩定的版本
        "tensorflow-federated==0.73.0",  # 與 TF 2.15 相容
        "tensorflow-privacy==0.9.0",  # 最新相容版本
        "tensorflow-estimator==2.15.0",  # 明確指定 estimator 版本
    ]

    for install_cmd in compatible_installs:
        print(f"📦 安裝 {install_cmd}")
        result = subprocess.run([
            sys.executable, "-m", "pip", "install", "--no-cache-dir",
            "--no-deps", install_cmd  # 使用 --no-deps 避免自動升級依賴
        ], capture_output=True, text=True)

        if result.returncode == 0:
            print(f"✅ {install_cmd} 安裝成功")
        else:
            print(f"❌ {install_cmd} 安裝失敗")
            print(f"   錯誤: {result.stderr}")

    # 步驟5: 安裝其他必要的依賴
    print("📦 安裝其他必要依賴...")
    other_deps = [
        "protobuf>=3.20,<4",
        "absl-py>=1.0.0",
        "attrs>=21.4.0",
        "cachetools>=5.2",
        "dm-tree>=0.1.8",
        "grpcio>=1.48.2",
        "jax>=0.4.1",
        "jaxlib>=0.4.1",
        "portpicker>=1.5.2",
        "semantic-version>=2.10",
        "sortedcontainers>=2.4.0",
        "tqdm>=4.64.1",
        "typing-extensions>=4.5.0"
    ]

    for dep in other_deps:
        subprocess.run([
            sys.executable, "-m", "pip", "install", "--no-cache-dir", dep
        ], capture_output=True, check=False)

# 執行安裝
install_compatible_federated_environment()

print("\n" + "="*60)
print("⚠️  重要提醒：請立即重新啟動 Runtime！")
print("   Runtime > Restart session")
print("   然後重新執行此 Cell 進行驗證")
print("="*60)

# =========================================================
# ❷ 驗證安裝結果
# =========================================================
print("\n🔍 驗證套件安裝...")

# 基礎模組
try:
    import json
    import numpy as np
    print(f"✅ NumPy {np.__version__} 載入成功")
    import pandas as pd
    from collections import OrderedDict
    from typing import List, Tuple, Dict, Any, Optional
    print("✅ 基礎模組導入成功")
except Exception as e:
    print(f"❌ 基礎模組導入失敗: {e}")

# TensorFlow 驗證
try:
    import tensorflow as tf
    print(f"✅ TensorFlow {tf.__version__} 載入成功")

    # 設定 TensorFlow 配置
    tf.get_logger().setLevel('ERROR')

    # GPU 設定
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU 已設定: {len(gpus)} 個 GPU 可用")
    else:
        print("📱 使用 CPU 模式")

except Exception as e:
    print(f"❌ TensorFlow 載入失敗: {e}")

# TensorFlow Federated 驗證
try:
    import tensorflow_federated as tff
    print(f"✅ TensorFlow Federated {tff.__version__} 載入成功")
except Exception as e:
    print(f"❌ TensorFlow Federated 載入失敗: {e}")
    print("💡 將使用自實現的聯邦學習核心")

# TensorFlow Privacy 驗證
try:
    import tensorflow_privacy as tfp
    print(f"✅ TensorFlow Privacy 載入成功")
except Exception as e:
    print(f"❌ TensorFlow Privacy 載入失敗: {e}")

# =========================================================
# ❸ 聯邦學習配置（簡化版，確保穩定性）
# =========================================================
FEDERATED_CONFIG = {
    # 基本參數
    "num_clients": 7,
    "base_stations": [1, 2, 3, 4, 5, 6, 7],
    "rounds": 30,  # 減少輪數確保穩定性
    "local_epochs": 3,
    "batch_size": 512,
    "learning_rate": 0.001,

    # 簡化設定避免相依性問題
    "dp_enabled": False,  # 暫時關閉差分隱私
    "secure_aggregation": False,
    "compression_enabled": False,
    "personalization": False,

    # 客戶端選擇
    "clients_per_round": 5,
    "min_available_clients": 3,

    # 差分隱私參數（如果啟用）
    "dp_noise_multiplier": 0.1,
    "dp_l2_norm_clip": 1.0
}

print(f"\n🏗️ 聯邦學習環境配置完成")
print(f"📊 配置: {FEDERATED_CONFIG['num_clients']} 客戶端, {FEDERATED_CONFIG['rounds']} 輪次")
print(f"🔒 隱私保護: {'啟用' if FEDERATED_CONFIG['dp_enabled'] else '關閉'}")

# =========================================================
# ❹ 自實現聯邦學習核心（備選方案）
# =========================================================
class SimpleFederatedLearning:
    """簡化的聯邦學習實現，避免複雜的套件相依性"""

    def __init__(self, config):
        self.config = config
        self.global_model_weights = None
        self.training_history = {
            "rounds": [],
            "avg_loss": [],
            "client_losses": []
        }

    def create_model(self, input_dim):
        """創建標準神經網路模型"""
        try:
            import tensorflow as tf

            model = tf.keras.Sequential([
                tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.3),

                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(0.2),

                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dropout(0.1),

                tf.keras.layers.Dense(1, activation='sigmoid')
            ])

            model.compile(
                optimizer=tf.keras.optimizers.Adam(self.config["learning_rate"]),
                loss='mse',
                metrics=['mae']
            )

            return model
        except Exception as e:
            print(f"❌ 模型創建失敗: {e}")
            return None

    def federated_averaging(self, client_weights_list, client_sizes):
        """實現聯邦平均算法"""
        if not client_weights_list:
            return None

        # 計算加權平均
        total_size = sum(client_sizes)
        averaged_weights = []

        for layer_idx in range(len(client_weights_list[0])):
            weighted_sum = None

            for client_weights, size in zip(client_weights_list, client_sizes):
                layer_weight = client_weights[layer_idx] * (size / total_size)

                if weighted_sum is None:
                    weighted_sum = layer_weight
                else:
                    weighted_sum += layer_weight

            averaged_weights.append(weighted_sum)

        return averaged_weights

    def train_client(self, client_id, X_train, y_train, X_val, y_val):
        """訓練單一客戶端"""
        # 創建本地模型
        model = self.create_model(X_train.shape[1])

        if model is None:
            return None, 0, {}

        # 如果有全域模型權重，則載入
        if self.global_model_weights is not None:
            model.set_weights(self.global_model_weights)

        # 本地訓練
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=self.config["local_epochs"],
            batch_size=self.config["batch_size"],
            verbose=0
        )

        return model.get_weights(), len(X_train), history.history

print("✅ 簡化聯邦學習類別定義完成")

# 創建聯邦學習系統
federated_system = SimpleFederatedLearning(FEDERATED_CONFIG)

print("\n🎉 聯邦學習環境準備完成！")
print("💡 如果 TensorFlow Federated 載入失敗，系統將使用自實現的聯邦學習核心")
print("📋 下一步：執行 Cell 2 進行資料準備")

# 檢查是否需要重啟
need_restart = False
try:
    import tensorflow as tf
    import numpy as np
    # 測試是否可以正常使用
    test_array = np.array([1, 2, 3])
    test_tensor = tf.constant(test_array)
    print("\n✅ 環境測試通過，可以繼續執行")
except Exception as e:
    print(f"\n❌ 環境測試失敗: {e}")
    print("⚠️  請重新啟動 Runtime 後再次執行此 Cell")
    need_restart = True

if not need_restart:
    print("\n🚀 可以直接執行 Cell 2，無需重啟 Runtime")

🚀 徹底重新安裝聯邦學習環境...
🗑️ 徹底移除 tensorflow-federated...
🗑️ 徹底移除 tensorflow-privacy...
🗑️ 徹底移除 tensorflow-estimator...
🗑️ 徹底移除 tensorflow-model-optimization...
🗑️ 徹底移除 tensorflow...
🗑️ 徹底移除 tf-keras...
🗑️ 徹底移除 numpy...
📦 安裝相容的 NumPy 版本...
✅ NumPy<2.0 安裝成功
📦 安裝相容的套件組合...
📦 安裝 tensorflow==2.15.0
✅ tensorflow==2.15.0 安裝成功
📦 安裝 tensorflow-federated==0.73.0
✅ tensorflow-federated==0.73.0 安裝成功
📦 安裝 tensorflow-privacy==0.9.0
✅ tensorflow-privacy==0.9.0 安裝成功
📦 安裝 tensorflow-estimator==2.15.0
✅ tensorflow-estimator==2.15.0 安裝成功
📦 安裝其他必要依賴...

⚠️  重要提醒：請立即重新啟動 Runtime！
   Runtime > Restart session
   然後重新執行此 Cell 進行驗證

🔍 驗證套件安裝...
✅ NumPy 1.26.4 載入成功
✅ 基礎模組導入成功
✅ TensorFlow 2.15.0 載入成功
📱 使用 CPU 模式
❌ TensorFlow Federated 載入失敗: No module named 'dp_accounting'
💡 將使用自實現的聯邦學習核心
❌ TensorFlow Privacy 載入失敗: No module named 'dp_accounting'

🏗️ 聯邦學習環境配置完成
📊 配置: 7 客戶端, 30 輪次
🔒 隱私保護: 關閉
✅ 簡化聯邦學習類別定義完成

🎉 聯邦學習環境準備完成！
💡 如果 TensorFlow Federated 載入失敗，系統將使用自實現的聯邦學習核心
📋 下一步：執行 Cell 2 進行資料準備

✅ 環境測試通過，可以繼續執行

🚀 可以直接執行 Ce

In [5]:
# @title Cell 2 | 聯邦資料分割與預處理
import hashlib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class FederatedDataProcessor:
    def __init__(self, config):
        self.config = config
        self.client_scalers = {}
        self.global_stats = {}

    def load_and_split_data(self):
        """載入資料並按基站分割為聯邦客戶端"""
        print("🔍 載入完整資料集...")

        # 載入處理後的特徵資料
        if os.path.exists("coloran_processed_features.parquet"):
            df = pd.read_parquet("coloran_processed_features.parquet")
            print(f"✅ 載入 {len(df):,} 筆記錄")
        else:
            raise FileNotFoundError("請先執行原始 Cell 3 生成特徵資料")

        # 載入特徵名稱
        if not os.path.exists("feature_metadata.json"):
             raise FileNotFoundError("請確認 feature_metadata.json 檔案存在")
        with open("feature_metadata.json", "r") as f:
            feature_names = json.load(f)["feature_names"]

        # 按基站分割資料
        client_data = {}
        data_stats = {}

        for i, bs_id in enumerate(self.config["base_stations"]):
            # 確保 bs_id 是整數型別以便比較
            df['bs_id'] = df['bs_id'].astype(int)
            client_df = df[df["bs_id"] == bs_id].copy()

            if len(client_df) == 0:
                print(f"⚠️ 基站 {bs_id} 無資料，跳過")
                continue

            # 提取特徵和目標
            X = client_df[feature_names].astype(np.float32).values
            y = client_df["allocation_efficiency"].astype(np.float32).values

            # 清理資料
            mask = np.isfinite(X).all(axis=1) & np.isfinite(y)
            X, y = X[mask], y[mask]

            # 分割訓練/測試集
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )

            # 本地標準化
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
            X_test_scaled = scaler.transform(X_test).astype(np.float32)

            client_data[f"client_{i}"] = {
                "bs_id": bs_id,
                "X_train": X_train_scaled, "y_train": y_train,
                "X_test": X_test_scaled, "y_test": y_test,
                "scaler": scaler, "feature_names": feature_names
            }

            data_stats[f"client_{i}"] = {"bs_id": bs_id, "train_samples": len(X_train), "test_samples": len(X_test)}
            print(f"📊 客戶端 {i} (BS-{bs_id}): {len(X_train):,} 訓練樣本, {len(X_test):,} 測試樣本")

        self.global_stats = {"total_clients": len(client_data), "client_stats": data_stats}
        print(f"\n🌐 聯邦資料分割完成，共 {self.global_stats['total_clients']} 個有效客戶端。")

        return client_data, feature_names

    def create_tf_datasets(self, client_data):
        """創建 TensorFlow Federated 資料集"""
        federated_train_data, federated_test_data = [], []
        for client_id, data in client_data.items():
            train_ds = tf.data.Dataset.from_tensor_slices({"x": data["X_train"], "y": data["y_train"]}).batch(self.config["batch_size"])
            test_ds = tf.data.Dataset.from_tensor_slices({"x": data["X_test"], "y": data["y_test"]}).batch(self.config["batch_size"])
            federated_train_data.append(train_ds)
            federated_test_data.append(test_ds)
        return federated_train_data, federated_test_data

# 執行資料處理
processor = FederatedDataProcessor(FEDERATED_CONFIG)
client_data, feature_names = processor.load_and_split_data()
federated_train_data, federated_test_data = processor.create_tf_datasets(client_data)

print("✅ 聯邦資料集準備完成!")


🔍 載入完整資料集...
✅ 載入 35,512,393 筆記錄
📊 客戶端 0 (BS-1): 4,039,936 訓練樣本, 1,009,985 測試樣本
📊 客戶端 1 (BS-2): 4,057,276 訓練樣本, 1,014,319 測試樣本
📊 客戶端 2 (BS-3): 4,130,032 訓練樣本, 1,032,509 測試樣本
📊 客戶端 3 (BS-4): 3,943,985 訓練樣本, 985,997 測試樣本


KeyboardInterrupt: 

In [ ]:
# @title Cell 3｜自實現高階聯邦學習框架（避免TFF相依性）
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import copy
from typing import List, Dict, Tuple, Optional
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# =========================================================
# ❶ 聯邦學習核心類別
# =========================================================
class FederatedSliceModel:
    """高階聯邦切片模型，支援個人化與差分隱私"""

    def __init__(self, input_dim: int, config: Dict):
        self.input_dim = input_dim
        self.config = config

    def create_model(self) -> tf.keras.Model:
        """創建支援個人化的神經網路模型"""
        inputs = tf.keras.layers.Input(shape=(self.input_dim,))

        # 共享層（參與聚合）
        x = tf.keras.layers.Dense(256, activation='relu', name='shared_dense_1')(inputs)
        x = tf.keras.layers.BatchNormalization(name='shared_bn_1')(x)
        x = tf.keras.layers.Dropout(0.3)(x)

        x = tf.keras.layers.Dense(128, activation='relu', name='shared_dense_2')(x)
        x = tf.keras.layers.BatchNormalization(name='shared_bn_2')(x)
        x = tf.keras.layers.Dropout(0.3)(x)

        # 個人化層（不參與聚合）
        if self.config.get("personalization", False):
            personal_branch = tf.keras.layers.Dense(64, activation='relu',
                                                   name='personal_layer')(x)
            personal_branch = tf.keras.layers.Dropout(0.2)(personal_branch)
            x = tf.keras.layers.Concatenate()([x, personal_branch])

        x = tf.keras.layers.Dense(32, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        outputs = tf.keras.layers.Dense(1, activation='sigmoid', dtype='float32')(x)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)

        # 編譯模型
        optimizer = tf.keras.optimizers.Adam(self.config["learning_rate"])
        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

        return model

class DifferentialPrivacyOptimizer:
    """差分隱私優化器"""

    def __init__(self, noise_multiplier: float, l2_norm_clip: float):
        self.noise_multiplier = noise_multiplier
        self.l2_norm_clip = l2_norm_clip

    def add_noise_to_gradients(self, gradients: List[tf.Tensor]) -> List[tf.Tensor]:
        """為梯度添加高斯噪聲"""
        noisy_gradients = []

        for grad in gradients:
            if grad is not None:
                # 梯度裁剪
                clipped_grad = tf.clip_by_norm(grad, self.l2_norm_clip)

                # 添加高斯噪聲
                noise = tf.random.normal(
                    shape=tf.shape(clipped_grad),
                    mean=0.0,
                    stddev=self.noise_multiplier * self.l2_norm_clip,
                    dtype=clipped_grad.dtype
                )

                noisy_grad = clipped_grad + noise
                noisy_gradients.append(noisy_grad)
            else:
                noisy_gradients.append(None)

        return noisy_gradients

class SecureAggregator:
    """安全聚合器，模擬同態加密與安全多方計算"""

    def __init__(self, num_clients: int):
        self.num_clients = num_clients
        self.client_secrets = {}

    def generate_client_secrets(self):
        """為每個客戶端生成密鑰"""
        for i in range(self.num_clients):
            self.client_secrets[i] = np.random.randint(0, 1000000, size=1)[0]

    def encrypt_weights(self, weights: List[np.ndarray], client_id: int) -> List[np.ndarray]:
        """模擬權重加密（簡化版同態加密）"""
        secret = self.client_secrets.get(client_id, 0)
        encrypted_weights = []

        for w in weights:
            # 簡化的加法同態加密模擬
            noise = np.random.normal(0, 0.001, w.shape) * secret / 1000000
            encrypted_w = w + noise
            encrypted_weights.append(encrypted_w)

        return encrypted_weights

    def secure_aggregate(self, encrypted_weights_list: List[List[np.ndarray]]) -> List[np.ndarray]:
        """安全聚合多個客戶端的加密權重"""
        if not encrypted_weights_list:
            return []

        # 初始化聚合結果
        aggregated_weights = []
        num_layers = len(encrypted_weights_list[0])

        for layer_idx in range(num_layers):
            layer_weights = [client_weights[layer_idx] for client_weights in encrypted_weights_list]

            # 加權平均（模擬安全聚合）
            aggregated_layer = np.mean(layer_weights, axis=0)
            aggregated_weights.append(aggregated_layer)

        return aggregated_weights

class AdvancedFederatedLearning:
    """高階聯邦學習系統"""

    def __init__(self, config: Dict):
        self.config = config
        self.global_model = None
        self.client_models = {}
        self.training_history = {
            "rounds": [],
            "global_loss": [],
            "client_losses": [],
            "privacy_budget": [],
            "communication_cost": [],
            "convergence_rate": []
        }

        # 初始化組件
        if config.get("dp_enabled", False):
            self.dp_optimizer = DifferentialPrivacyOptimizer(
                config["dp_noise_multiplier"],
                config["dp_l2_norm_clip"]
            )
        else:
            self.dp_optimizer = None

        if config.get("secure_aggregation", False):
            self.secure_aggregator = SecureAggregator(config["num_clients"])
            self.secure_aggregator.generate_client_secrets()
        else:
            self.secure_aggregator = None

    def initialize_global_model(self, input_dim: int):
        """初始化全域模型"""
        model_builder = FederatedSliceModel(input_dim, self.config)
        self.global_model = model_builder.create_model()
        print(f"✅ 全域模型初始化完成 - 參數數量: {self.global_model.count_params():,}")

    def create_client_data(self, df: pd.DataFrame, feature_names: List[str]) -> Dict:
        """創建客戶端資料分割（Non-IID）"""
        print("🔄 創建Non-IID聯邦資料分割...")

        client_data = {}

        for i, bs_id in enumerate(self.config["base_stations"]):
            # 按基站分割資料
            client_df = df[df["bs_id"] == bs_id].copy()

            if len(client_df) == 0:
                print(f"⚠️ 基站 {bs_id} 無資料")
                continue

            # 創建Non-IID特性：不同基站專注不同切片類型
            slice_preference = i % 3  # 0: eMBB, 1: URLLC, 2: mMTC

            # 80% 偏好切片 + 20% 其他切片
            if 'slice_id' in client_df.columns:
                preferred_data = client_df[client_df['slice_id'] == slice_preference]
                other_data = client_df[client_df['slice_id'] != slice_preference]

                if len(preferred_data) > 0 and len(other_data) > 0:
                    n_preferred = min(len(preferred_data), int(0.8 * min(50000, len(client_df))))
                    n_other = min(len(other_data), int(0.2 * min(50000, len(client_df))))

                    selected_data = pd.concat([
                        preferred_data.sample(n=n_preferred, random_state=42+i),
                        other_data.sample(n=n_other, random_state=42+i)
                    ])
                else:
                    selected_data = client_df.sample(n=min(50000, len(client_df)), random_state=42+i)
            else:
                selected_data = client_df.sample(n=min(50000, len(client_df)), random_state=42+i)

            # 提取特徵和目標
            X = selected_data[feature_names].astype(np.float32).values
            y = selected_data["allocation_efficiency"].astype(np.float32).values

            # 清理資料
            mask = np.isfinite(X).all(axis=1) & np.isfinite(y)
            X, y = X[mask], y[mask]

            if len(X) < 100:
                print(f"⚠️ 客戶端 {i} 資料不足")
                continue

            # 分割訓練/測試集
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )

            # 標準化
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
            X_test_scaled = scaler.transform(X_test).astype(np.float32)

            client_data[f"client_{i}"] = {
                "client_id": i,
                "bs_id": bs_id,
                "slice_preference": slice_preference,
                "X_train": X_train_scaled,
                "y_train": y_train,
                "X_test": X_test_scaled,
                "y_test": y_test,
                "scaler": scaler,
                "data_size": len(X_train)
            }

            print(f"📊 客戶端 {i} (BS-{bs_id}): {len(X_train):,} 訓練, {len(X_test):,} 測試, 偏好切片={slice_preference}")

        return client_data

    def federated_averaging(self, client_weights_list: List[List[np.ndarray]],
                          client_sizes: List[int]) -> List[np.ndarray]:
        """FedAvg算法實現"""
        if not client_weights_list:
            return []

        total_size = sum(client_sizes)
        averaged_weights = []

        # 計算加權平均
        for layer_idx in range(len(client_weights_list[0])):
            weighted_sum = None

            for client_weights, size in zip(client_weights_list, client_sizes):
                layer_weight = client_weights[layer_idx] * (size / total_size)

                if weighted_sum is None:
                    weighted_sum = layer_weight
                else:
                    weighted_sum += layer_weight

            averaged_weights.append(weighted_sum)

        return averaged_weights

    def train_client(self, client_id: str, client_data: Dict) -> Tuple[List[np.ndarray], Dict]:
        """訓練單一客戶端"""
        # 創建客戶端模型
        model_builder = FederatedSliceModel(client_data["X_train"].shape[1], self.config)
        client_model = model_builder.create_model()

        # 從全域模型載入權重
        if self.global_model is not None:
            client_model.set_weights(self.global_model.get_weights())

        # 本地訓練
        history = client_model.fit(
            client_data["X_train"],
            client_data["y_train"],
            validation_data=(client_data["X_test"], client_data["y_test"]),
            epochs=self.config["local_epochs"],
            batch_size=self.config["batch_size"],
            verbose=0
        )

        # 獲取訓練後的權重
        trained_weights = client_model.get_weights()

        # 應用差分隱私（如果啟用）
        if self.dp_optimizer is not None:
            # 計算梯度並添加噪聲
            with tf.GradientTape() as tape:
                predictions = client_model(client_data["X_train"], training=True)
                loss = tf.keras.losses.MeanSquaredError()(client_data["y_train"], predictions)

            gradients = tape.gradient(loss, client_model.trainable_variables)
            noisy_gradients = self.dp_optimizer.add_noise_to_gradients(gradients)

            # 應用噪聲梯度更新
            for i, (weight, grad) in enumerate(zip(trained_weights, noisy_gradients)):
                if grad is not None:
                    trained_weights[i] = weight - self.config["learning_rate"] * grad.numpy()

        return trained_weights, {
            "loss": history.history["loss"][-1],
            "val_loss": history.history["val_loss"][-1],
            "data_size": client_data["data_size"]
        }

    def run_federated_training(self, client_data: Dict):
        """執行聯邦學習訓練"""
        print(f"\n🚀 開始聯邦學習訓練...")
        print(f"📊 配置: {len(client_data)} 個客戶端, {self.config['rounds']} 輪")

        # 初始化全域模型
        sample_client = next(iter(client_data.values()))
        self.initialize_global_model(sample_client["X_train"].shape[1])

        for round_num in range(1, self.config["rounds"] + 1):
            print(f"\n🔄 第 {round_num}/{self.config['rounds']} 輪")

            # 客戶端選擇
            available_clients = list(client_data.keys())
            if len(available_clients) > self.config.get("clients_per_round", len(available_clients)):
                selected_clients = np.random.choice(
                    available_clients,
                    size=self.config["clients_per_round"],
                    replace=False
                )
            else:
                selected_clients = available_clients

            # 並行訓練客戶端
            client_weights_list = []
            client_sizes = []
            round_losses = []

            for client_id in selected_clients:
                try:
                    weights, metrics = self.train_client(client_id, client_data[client_id])

                    # 安全聚合（如果啟用）
                    if self.secure_aggregator is not None:
                        client_idx = int(client_id.split('_')[1])
                        weights = self.secure_aggregator.encrypt_weights(weights, client_idx)

                    client_weights_list.append(weights)
                    client_sizes.append(metrics["data_size"])
                    round_losses.append(metrics["loss"])

                except Exception as e:
                    print(f"⚠️ 客戶端 {client_id} 訓練失敗: {e}")

            # 聚合權重
            if client_weights_list:
                if self.secure_aggregator is not None:
                    aggregated_weights = self.secure_aggregator.secure_aggregate(client_weights_list)
                else:
                    aggregated_weights = self.federated_averaging(client_weights_list, client_sizes)

                # 更新全域模型
                self.global_model.set_weights(aggregated_weights)

                # 評估全域模型
                global_metrics = self.evaluate_global_model(client_data)

                # 記錄歷史
                self.training_history["rounds"].append(round_num)
                self.training_history["global_loss"].append(global_metrics["loss"])
                self.training_history["client_losses"].append(np.mean(round_losses))

                # 計算隱私預算（如果啟用DP）
                if self.dp_optimizer is not None:
                    privacy_budget = round_num * self.config["dp_noise_multiplier"]
                    self.training_history["privacy_budget"].append(privacy_budget)

                # 計算通訊成本
                model_size = sum(np.prod(w.shape) for w in aggregated_weights)
                comm_cost = model_size * len(selected_clients)
                self.training_history["communication_cost"].append(comm_cost)

                # 進度報告
                if round_num % 5 == 0:
                    print(f"   全域損失: {global_metrics['loss']:.6f}")
                    print(f"   平均客戶端損失: {np.mean(round_losses):.6f}")
                    print(f"   參與客戶端: {len(selected_clients)}")

                    if self.dp_optimizer is not None:
                        print(f"   隱私預算 (ε): {privacy_budget:.2f}")

            # 早停檢查
            if round_num > 10:
                recent_losses = self.training_history["global_loss"][-5:]
                if len(recent_losses) == 5:
                    improvement = (recent_losses[0] - recent_losses[-1]) / recent_losses[0]
                    if improvement < 0.001:  # 改善小於0.1%
                        print(f"🛑 早停：改善幅度過小 ({improvement:.4f})")
                        break

        print("✅ 聯邦學習訓練完成！")
        return self.training_history

    def evaluate_global_model(self, client_data: Dict) -> Dict:
        """評估全域模型在所有客戶端上的效能"""
        total_loss = 0
        total_samples = 0

        for client_info in client_data.values():
            if len(client_info["X_test"]) > 0:
                loss, _ = self.global_model.evaluate(
                    client_info["X_test"],
                    client_info["y_test"],
                    verbose=0
                )
                samples = len(client_info["X_test"])
                total_loss += loss * samples
                total_samples += samples

        avg_loss = total_loss / total_samples if total_samples > 0 else float('inf')
        return {"loss": avg_loss}

# =========================================================
# ❷ 執行聯邦學習
# =========================================================
def run_advanced_federated_learning():
    """執行高階聯邦學習主程序"""
    print("🚀 啟動高階聯邦學習系統...")

    # 載入資料
    if os.path.exists("coloran_processed_features.parquet"):
        df = pd.read_parquet("coloran_processed_features.parquet")
        print(f"✅ 載入資料: {len(df):,} 筆記錄")
    else:
        print("❌ 找不到處理後的特徵資料")
        return None

    # 載入特徵名稱
    with open("feature_metadata.json", "r") as f:
        feature_names = json.load(f)["feature_names"]

    # 創建聯邦學習系統
    fed_system = AdvancedFederatedLearning(FEDERATED_CONFIG)

    # 創建客戶端資料
    client_data = fed_system.create_client_data(df, feature_names)

    if not client_data:
        print("❌ 無法創建客戶端資料")
        return None

    # 執行聯邦訓練
    history = fed_system.run_federated_training(client_data)

    # 保存結果
    results = {
        "config": FEDERATED_CONFIG,
        "history": history,
        "timestamp": datetime.now().isoformat()
    }

    with open("federated_learning_results.json", "w") as f:
        json.dump(results, f, indent=2, default=str)

    # 保存模型
    fed_system.global_model.save("federated_global_model.h5")

    print("💾 結果已保存到 federated_learning_results.json")
    print("💾 全域模型已保存到 federated_global_model.h5")

    return fed_system, history

# 執行聯邦學習
if __name__ == "__main__":
    federated_system, training_history = run_advanced_federated_learning()
    print("🎉 高階聯邦學習完成！")


In [ ]:
# @title Cell 4｜A100 GPU 加速機器學習訓練器（網路切片資源分配優化）
import os, json, warnings, joblib, gc
import numpy as np
import pandas as pd
from datetime import datetime
import subprocess
warnings.filterwarnings("ignore")
np.random.seed(42)

# ========= A. 安裝必要套件 & GPU 偵測 =========
def install_required_packages():
    """安裝必要的 GPU 套件（適用於 A100）"""
    print("🔄 檢查必要套件...")
    try:
        import tensorflow as tf
        print(f"✓ TensorFlow {tf.__version__} 已安裝")
    except ImportError:
        print("⚠️ 安裝 TensorFlow...")
        !pip install -q tensorflow

    try:
        import cuml
        print(f"✓ cuML {cuml.__version__} 已安裝")
    except ImportError:
        print("⚠️ 安裝 cuML...")
        # A100 環境需要的特定版本
        !pip install -q cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
        # 重新導入
        try:
            import cuml
            print(f"✓ cuML {cuml.__version__} 安裝成功")
        except ImportError:
            print("❌ cuML 安裝失敗，將使用 CPU 備選方案")

def setup_gpu_environment():
    """設定 GPU 環境並檢查可用性"""
    gpu = {"gpu_available": False, "cuml_available": False, "tf_gpu": False}

    # 檢查 NVIDIA GPU
    try:
        result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
        if result.returncode == 0:
            gpu["gpu_available"] = True
            print("✅ NVIDIA GPU 已偵測到")
            # 顯示 GPU 資訊
            for line in result.stdout.split('\n')[:10]:
                if "A100" in line:
                    print(f"  {line.strip()}")
    except FileNotFoundError:
        print("⚠️ nvidia-smi 未找到")

    # 設定 TensorFlow
    if gpu["gpu_available"]:
        try:
            import tensorflow as tf
            gpus = tf.config.list_physical_devices("GPU")
            if gpus:
                for g in gpus:
                    tf.config.experimental.set_memory_growth(g, True)
                gpu["tf_gpu"] = True
                print(f"✅ TensorFlow 已啟用 {len(gpus)} 個 GPU")
                # 設定 GPU 記憶體限制
                tf.config.experimental.set_virtual_device_configuration(
                    gpus[0],
                    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=32*1024)]
                )
                print("  已設定 GPU 記憶體限制：32GB")
        except Exception as e:
            print(f"⚠️ TensorFlow GPU 初始化失敗: {e}")

    # 檢查 cuML
    try:
        import cuml
        from cuml.common import logger
        logger.set_level(logger.level_enum.info)
        gpu["cuml_available"] = True
        print("✅ cuML 已啟用 - 將使用 GPU 加速隨機森林")
    except ImportError:
        print("⚠️ cuML 未找到 - 將使用 CPU 隨機森林 (多執行緒)")
    except Exception as e:
        print(f"⚠️ cuML 初始化失敗: {e}")

    return gpu

# 安裝必要套件並設定 GPU
install_required_packages()
GPU_INFO = setup_gpu_environment()

# ========= B. 資料載入 & 採樣優化 =========
def load_processed_features(max_rows=None):
    """載入特徵資料並進行智慧型採樣"""
    candidates = [
        "coloran_processed_features.parquet",
        "oran_processed_features.parquet",
        "coloran_complete_dataset.parquet",
    ]

    for p in candidates:
        if os.path.exists(p):
            # 檢查檔案大小
            file_size_gb = os.path.getsize(p) / (1024**3)
            print(f"📁 找到資料檔案: {p} ({file_size_gb:.2f} GB)")

            # 大於 1GB 且 max_rows 有設定時，使用採樣
            if file_size_gb > 1 and max_rows:
                # 讀取列數
                file_row_count = pd.read_parquet(p, columns=[]).shape[0]
                print(f"  總列數: {file_row_count:,}")

                if file_row_count > max_rows:
                    # 計算採樣比例
                    fraction = max_rows / file_row_count
                    print(f"  使用採樣 {fraction:.2%} 來載入資料...")
                    df = pd.read_parquet(p, engine='pyarrow').sample(frac=fraction, random_state=42)
                else:
                    df = pd.read_parquet(p, engine='pyarrow')
            else:
                df = pd.read_parquet(p, engine='pyarrow')

            # 載入特徵名稱
            try:
                with open("feature_metadata.json", "r") as f:
                    meta = json.load(f)
                feats = meta["feature_names"]
            except FileNotFoundError:
                # 使用預設特徵名稱
                feats = ['num_ues', 'slice_id', 'sched_policy_num', 'allocated_rbgs',
                        'bs_id', 'exp_id', 'sum_requested_prbs', 'sum_granted_prbs',
                        'prb_utilization', 'throughput_efficiency', 'qos_score',
                        'network_load', 'hour', 'minute', 'day_of_week']

            print(f"✅ 載入完成: {len(df):,} 列 × {len(feats)} 特徵")
            return df, feats

    print("❌ 找不到特徵檔案")
    return None, None

# ========= C. A100 最佳化預測器 =========
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

class A100OptimizedPredictor:
    def __init__(self, df, feats):
        self.df, self.feats = df, feats
        self.scaler = self.rf_model = self.nn_model = None

        # A100 優化參數
        self.rf_batch_size = 500000  # 隨機森林批次大小
        self.nn_batch_size = 4096    # A100 適合的批次大小

        # 自動採樣設定
        self.use_sampling = len(df) > 10_000_000  # 超過1千萬列才採樣
        if self.use_sampling:
            self.sample_size = min(5_000_000, len(df))  # 最多500萬列
            print(f"🔄 大型資料集 ({len(df):,} 列) - 將使用採樣 {self.sample_size:,} 列")

    # --- 資料準備 ---
    def prepare_data(self):
        print("📊 準備訓練資料...")

        # 智慧採樣
        if self.use_sampling:
            sample = self.df.sample(n=self.sample_size, random_state=42)
            X = sample[self.feats].astype(np.float32).values
            y = sample["allocation_efficiency"].astype(np.float32).values
        else:
            X = self.df[self.feats].astype(np.float32).values
            y = self.df["allocation_efficiency"].astype(np.float32).values

        # 檢查資料
        print(f"  特徵範圍: {X.min():.4f} 到 {X.max():.4f}")
        print(f"  目標範圍: {y.min():.4f} 到 {y.max():.4f}")

        # 分割資料
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=.2, random_state=42, shuffle=True
        )

        # 標準化 - 使用 float32 節省 GPU 記憶體
        self.scaler = StandardScaler()
        X_train = self.scaler.fit_transform(X_train).astype(np.float32)
        X_test = self.scaler.transform(X_test).astype(np.float32)

        print(f"✅ 訓練集: {X_train.shape}, 測試集: {X_test.shape}")
        return X_train, X_test, y_train, y_test

    # --- A100 最佳化隨機森林 ---
    def train_random_forest(self, X_train, y_train):
        print("\n🌲 訓練隨機森林...")

        if GPU_INFO["cuml_available"]:
            print("  使用 cuML GPU 隨機森林")
            try:
                from cuml.ensemble import RandomForestRegressor as cuRF
                self.rf_model = cuRF(
                    n_estimators=300,
                    max_depth=16,
                    random_state=42,
                    max_features=0.5  # A100 記憶體優化
                )
                self.rf_model.fit(X_train, y_train)
                print("✅ GPU 隨機森林訓練完成")
            except Exception as e:
                print(f"⚠️ GPU 隨機森林失敗: {e}")
                print("  退回使用 CPU 版本")
                GPU_INFO["cuml_available"] = False

        # 如果 GPU 不可用或失敗，使用 CPU
        if not GPU_INFO["cuml_available"]:
            print("  使用 CPU 隨機森林 (sklearn 多執行緒)")
            from sklearn.ensemble import RandomForestRegressor

            # 使用大量執行緒 & 記憶體優化參數
            cpu_count = os.cpu_count() or 4
            print(f"  CPU 執行緒數: {cpu_count}")

            self.rf_model = RandomForestRegressor(
                n_estimators=200,
                max_depth=12,
                min_samples_split=10,
                min_samples_leaf=8,
                random_state=42,
                n_jobs=cpu_count,
                verbose=1
            )

            # 如果資料集太大，批次訓練
            if len(X_train) > self.rf_batch_size:
                print(f"  使用批次訓練: {self.rf_batch_size:,} 列/批次")
                indices = np.arange(len(X_train))
                np.random.shuffle(indices)
                batch_indices = indices[:self.rf_batch_size]
                self.rf_model.fit(X_train[batch_indices], y_train[batch_indices])
            else:
                self.rf_model.fit(X_train, y_train)

        # 生成特徵重要性
        if hasattr(self.rf_model, "feature_importances_"):
            fi = pd.DataFrame({
                "feature": self.feats,
                "importance": self.rf_model.feature_importances_
            }).sort_values("importance", ascending=False)

            # 儲存特徵重要性
            fi.to_csv("feature_importance.csv", index=False)
            print("💾 feature_importance.csv 已儲存")

            # 顯示前 5 個重要特徵
            print("\n📊 特徵重要性 Top-5:")
            for i, row in fi.head(5).iterrows():
                print(f"  {row['feature']}: {row['importance']:.4f}")

            return fi
        return None

    # --- A100 最佳化神經網路 ---
    def train_neural_net(self, X_train, y_train, X_test, y_test):
        print("\n🧠 訓練神經網路...")

        import tensorflow as tf
        tf.keras.backend.clear_session()

        # 檢查是否有 GPU
        if GPU_INFO["tf_gpu"]:
            print("  使用 GPU 加速訓練")

            # 啟用混合精度 (提升 A100 效能)
            try:
                from tensorflow.keras import mixed_precision
                policy = mixed_precision.Policy('mixed_float16')
                mixed_precision.set_global_policy(policy)
                print("  已啟用混合精度 (mixed_float16)")
            except:
                print("  未能啟用混合精度")
        else:
            print("  使用 CPU 訓練 (較慢)")

        # 建立模型
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu', input_shape=(len(self.feats),)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(0.2),

            # 輸出層使用 float32 避免混合精度問題
            tf.keras.layers.Dense(1, activation='linear', dtype='float32')
        ])

        # 編譯
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss='mse',
            metrics=['mae']
        )

        # 回調函數
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-6
            )
        ]

        # 訓練
        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=100,
            batch_size=self.nn_batch_size,
            callbacks=callbacks,
            verbose=1
        )

        self.nn_model = model
        return history

    # --- 評估 ---
    def evaluate(self, X_test, y_test):
        print("\n📈 評估模型效能...")
        results = {}

        if self.rf_model is not None:
            # 批次預測以避免 OOM
            if len(X_test) > 1000000:
                print("  使用批次預測評估隨機森林")
                batch_size = 500000
                predictions = []
                for i in range(0, len(X_test), batch_size):
                    end = min(i + batch_size, len(X_test))
                    batch_pred = self.rf_model.predict(X_test[i:end])
                    predictions.extend(batch_pred)
                p = np.array(predictions)
            else:
                p = self.rf_model.predict(X_test)

            results["Random Forest"] = {
                "R2": r2_score(y_test, p),
                "MAE": mean_absolute_error(y_test, p),
                "MSE": mean_squared_error(y_test, p)
            }

        if self.nn_model is not None:
            p = self.nn_model.predict(X_test, verbose=0).flatten()
            results["Neural Network"] = {
                "R2": r2_score(y_test, p),
                "MAE": mean_absolute_error(y_test, p),
                "MSE": mean_squared_error(y_test, p)
            }

        # 顯示結果
        for model_name, metrics in results.items():
            print(f"\n{model_name} 評估結果:")
            for metric, value in metrics.items():
                print(f"  {metric}: {value:.6f}")

        return results

# ========= D. 執行管道 =========
def run_gpu_ml_pipeline():
    print("🚀 啟動 A100 GPU 機器學習管道...")
    print("=" * 60)

    # 載入資料 (如果記憶體有限制，設定 max_rows)
    df, feats = load_processed_features(max_rows=5000000)
    if df is None:
        return None, None

    # 建立預測器
    predictor = A100OptimizedPredictor(df, feats)

    # 準備資料
    Xtr, Xte, ytr, yte = predictor.prepare_data()

    # 訓練隨機森林
    start_time = datetime.now()
    fi = predictor.train_random_forest(Xtr, ytr)
    rf_time = (datetime.now() - start_time).total_seconds()
    print(f"⏱️ 隨機森林訓練時間: {rf_time:.1f} 秒")

    # 訓練神經網路
    start_time = datetime.now()
    history = predictor.train_neural_net(Xtr, ytr, Xte, yte)
    nn_time = (datetime.now() - start_time).total_seconds()
    print(f"⏱️ 神經網路訓練時間: {nn_time:.1f} 秒")

    # 評估模型
    results = predictor.evaluate(Xte, yte)

    # 寫入 summary
    summary = {
        "timestamp": datetime.now().isoformat(),
        "data_size": len(df),
        "features": len(feats),
        "results": {k: {kk: float(vv) for kk, vv in v.items()}
                   for k, v in results.items()},
        "training_time": {
            "random_forest": rf_time,
            "neural_network": nn_time
        },
        "history_loss": [float(x) for x in history.history.get("loss", [])],
        "history_val_loss": [float(x) for x in history.history.get("val_loss", [])]
    }

    with open("training_summary_fixed.json", "w") as f:
        json.dump(summary, f, indent=2)
    print("💾 training_summary_fixed.json 已更新")

    # 儲存模型
    joblib.dump(predictor.rf_model, "fixed_rf_model.pkl")
    joblib.dump(predictor.scaler, "fixed_scaler.pkl")
    predictor.nn_model.save("fixed_nn_model.h5")
    print("✅ 模型與縮放器已儲存")

    # 釋放記憶體
    gc.collect()

    # 將結果設為全域變數以供後續使用
    global predictor_global, results_global
    predictor_global, results_global = predictor, results

    return predictor, results

# 直接執行
if __name__ == "__main__":
    predictor, results = run_gpu_ml_pipeline()

print("🎉 Cell 4 執行完成！")


In [ ]:
# @title Cell 5｜高效動態切片資源分配（修正瓶頸版）
import os, json, warnings, joblib, gc, time
import pandas as pd
import numpy as np
from datetime import datetime
warnings.filterwarnings("ignore")
np.random.seed(42)

class EfficientSliceAllocator:
    """高效率動態分配器 - 解決瓶頸問題"""

    def __init__(self, predictor, cfg, total_rbgs=17):
        self.predictor = predictor
        self.cfg = cfg
        self.total_rbgs = total_rbgs

        # 優化參數
        self.max_batch_size = 1000  # 減少批次大小
        self.timeout_s = 300  # 減少超時時間

        # 載入特徵名稱
        try:
            with open("feature_metadata.json", "r") as f:
                self.features = json.load(f)["feature_names"]
        except:
            self.features = ['num_ues','slice_id','sched_policy_num','allocated_rbgs',
                           'bs_id','exp_id','sum_requested_prbs','sum_granted_prbs',
                           'prb_utilization','throughput_efficiency','qos_score',
                           'network_load','hour','minute','day_of_week']

        print(f"🚀 高效分配器初始化完成 - {len(self.features)} 特徵")

    def _predict_single_allocation(self, state, allocation):
        """預測單一分配方案的效率 - 簡化版本"""
        # 只計算加權平均效率，不分別預測每個切片
        feature_vector = [
            state['num_ues'],
            1,  # 使用平均切片ID
            state['sched_policy'],
            np.mean(allocation),  # 平均分配
            state['bs_id'],
            state['exp_id'],
            state['requested_prbs'],
            np.sum(allocation),
            min(1.0, state['requested_prbs'] / max(1, np.sum(allocation))),
            state['dl_cqi'] / 15.0,
            (state['dl_cqi'] / 15.0 + state['ul_sinr'] / 30.0) / 2.0,
            state['num_ues'] / 42.0,
            state['hour'],
            state['minute'],
            state['day_of_week']
        ]

        # 單次預測
        X = np.array([feature_vector], dtype=np.float32)
        X_scaled = self.predictor.scaler.transform(X)
        prediction = self.predictor.rf_model.predict(X_scaled)[0]

        # 加入分配平衡度獎勵
        balance_penalty = np.std(allocation) * 0.01  # 懲罰不平衡分配
        return np.clip(prediction - balance_penalty, 0.0, 1.0)

    def optimized_exhaustive(self, state):
        """優化的窮舉搜索 - 減少搜索空間"""
        best_alloc = [6, 6, 5]  # 預設分配
        best_eff = self._predict_single_allocation(state, best_alloc)

        # 限制搜索範圍 - 只搜索合理的分配
        search_space = [
            [4, 8, 5], [5, 7, 5], [6, 6, 5], [7, 5, 5], [8, 4, 5],
            [5, 6, 6], [6, 5, 6], [7, 4, 6], [4, 7, 6], [5, 5, 7],
            [6, 4, 7], [4, 6, 7], [3, 8, 6], [8, 3, 6], [9, 4, 4]
        ]

        count = 0
        for alloc in search_space:
            if sum(alloc) == self.total_rbgs:
                eff = self._predict_single_allocation(state, alloc)
                if eff > best_eff:
                    best_eff = eff
                    best_alloc = alloc
                count += 1

        return best_alloc, best_eff

    def optimized_genetic(self, state, pop_size=20, generations=8):
        """優化的遺傳演算法 - 減少計算量"""
        population = [self._rand_alloc() for _ in range(pop_size)]

        for gen in range(generations):
            # 計算適應度
            fitness_scores = []
            for alloc in population:
                fitness_scores.append(self._predict_single_allocation(state, alloc))

            fitness_scores = np.array(fitness_scores)

            # 選擇最好的50%作為父母
            sorted_indices = np.argsort(fitness_scores)[::-1]
            elite_size = pop_size // 2
            elite_population = [population[i] for i in sorted_indices[:elite_size]]

            # 生成新族群
            new_population = elite_population.copy()  # 保留精英

            # 填充剩餘位置
            while len(new_population) < pop_size:
                if len(elite_population) >= 2:
                    p1, p2 = np.random.choice(elite_population, 2, replace=False).tolist()
                    child = self._crossover(p1, p2)
                    if np.random.rand() < 0.2:  # 20%變異率
                        child = self._mutate(child)
                    new_population.append(child)
                else:
                    new_population.append(self._rand_alloc())

            population = new_population

        # 找到最佳解
        final_fitness = [self._predict_single_allocation(state, alloc) for alloc in population]
        best_idx = np.argmax(final_fitness)
        return population[best_idx], final_fitness[best_idx]

    def _rand_alloc(self):
        """生成隨機分配"""
        alloc = [1, 1, 1]
        remaining = self.total_rbgs - 3
        for _ in range(remaining):
            alloc[np.random.randint(3)] += 1
        return alloc

    def _crossover(self, p1, p2):
        """交叉操作"""
        cut_point = np.random.randint(1, 3)
        child = p1[:cut_point] + p2[cut_point:]
        return self._repair_allocation(child)

    def _mutate(self, alloc):
        """變異操作"""
        alloc = alloc.copy()
        i, j = np.random.choice(3, 2, replace=False)
        if alloc[i] > 1:
            alloc[i] -= 1
            alloc[j] += 1
        return alloc

    def _repair_allocation(self, alloc):
        """修復分配"""
        alloc = [max(1, int(x)) for x in alloc]
        diff = self.total_rbgs - sum(alloc)
        if diff != 0:
            alloc[0] += diff  # 調整第一個切片
        return [max(1, x) for x in alloc]

    def simulate(self, steps=50, method="genetic"):
        """執行模擬"""
        optimizer_fn = self.optimized_genetic if method == "genetic" else self.optimized_exhaustive
        records = []
        start_time = time.time()

        print(f"🎯 開始 {method} 模擬 ({steps} 步)...")

        for t in range(steps):
            if time.time() - start_time > self.timeout_s:
                print(f"⏰ 超時停止，已完成 {t}/{steps} 步")
                break

            # 生成網路狀態
            current_state = {
                'num_ues': np.random.randint(5, 25),
                'sched_policy': np.random.randint(3),
                'requested_prbs': np.random.randint(8, 20),
                'dl_cqi': np.random.uniform(7, 13),
                'ul_sinr': np.random.uniform(10, 25),
                'hour': (t // 4) % 24,
                'minute': (t % 4) * 15,
                'day_of_week': t % 7,
                'bs_id': np.random.choice([1, 8, 15, 22, 29, 36, 43]),
                'exp_id': np.random.randint(1, 8)
            }

            # 基準分配
            baseline_alloc = [6, 6, 5]
            baseline_eff = self._predict_single_allocation(current_state, baseline_alloc)

            # 優化分配
            optimal_alloc, optimal_eff = optimizer_fn(current_state)

            improvement = optimal_eff - baseline_eff
            records.append({
                'step': t,
                'improve': improvement,
                'alloc': optimal_alloc,
                'baseline_eff': baseline_eff,
                'optimal_eff': optimal_eff
            })

            # 進度報告
            if (t + 1) % 10 == 0:
                elapsed = time.time() - start_time
                avg_improvement = np.mean([r['improve'] for r in records])
                print(f"   步驟 {t+1}/{steps} | 耗時: {elapsed:.1f}s | 平均改善: {avg_improvement:.4f}")

        total_time = time.time() - start_time
        avg_improvement = np.mean([r['improve'] for r in records]) if records else 0
        print(f"✅ {method} 模擬完成！耗時: {total_time:.1f}s | 最終平均改善: {avg_improvement:.4f}")

        return pd.DataFrame(records)

# 執行優化版本
def run_efficient_allocation():
    """執行高效動態分配"""
    # 載入模型
    if "predictor" in globals() and globals()["predictor"] is not None:
        pred = globals()["predictor"]
    else:
        raise FileNotFoundError("請先執行 Cell 4 訓練模型")

    # 載入配置
    try:
        with open("slice_configs.json", "r") as f:
            cfg = json.load(f)
    except:
        cfg = {}

    # 創建分配器
    allocator = EfficientSliceAllocator(pred, cfg)

    # 執行兩種方法的比較
    print("\n" + "="*60)
    results_exhaustive = allocator.simulate(40, "exhaustive")

    print("\n" + "="*60)
    results_genetic = allocator.simulate(50, "genetic")

    # 保存結果
    results_exhaustive.to_csv("efficient_exhaustive_results.csv", index=False)
    results_genetic.to_csv("efficient_genetic_results.csv", index=False)

    print(f"\n📊 結果比較:")
    print(f"窮舉法平均改善: {results_exhaustive['improve'].mean():.6f}")
    print(f"遺傳演算法平均改善: {results_genetic['improve'].mean():.6f}")

    return results_exhaustive, results_genetic

# 執行
if __name__ == "__main__":
    results_exh, results_gen = run_efficient_allocation()
    print("🎉 高效動態資源分配完成！")


In [ ]:
# @title Cell 6 | 聯邦學習結果視覺化與性能比較
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = (15, 10)

class FederatedResultsVisualizer:
    def __init__(self, training_results, allocation_results):
        self.training_results = training_results
        self.allocation_results = allocation_results

    def create_comprehensive_visualization(self):
        """創建聯邦學習完整視覺化"""
        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        fig.suptitle('Federated Learning for Dynamic Slice Resource Allocation - Comprehensive Results',
                    fontsize=16, fontweight='bold')

        # 1. 聯邦學習損失曲線
        self._plot_federated_training_loss(axes[0, 0])

        # 2. 通訊成本分析
        self._plot_communication_cost(axes[0, 1])

        # 3. 各基站效率提升比較
        self._plot_bs_efficiency_comparison(axes[0, 2])

        # 4. 聯邦 vs 集中式比較
        self._plot_federated_vs_centralized(axes[1, 0])

        # 5. 資源分配模式分析
        self._plot_allocation_patterns(axes[1, 1])

        # 6. 隱私保護效果
        self._plot_privacy_utility_tradeoff(axes[1, 2])

        plt.tight_layout()
        plt.savefig('federated_comprehensive_results.png', dpi=300, bbox_inches='tight')
        plt.show()

    def _plot_federated_training_loss(self, ax):
        """繪製聯邦學習訓練損失"""
        rounds = self.training_results["training_history"]["rounds"]
        train_loss = self.training_results["training_history"]["train_loss"]
        test_loss = self.training_results["training_history"]["test_loss"]

        ax.plot(rounds, train_loss, 'b-', linewidth=2, label='Federated Train Loss')
        ax.plot(rounds, test_loss, 'r-', linewidth=2, label='Federated Test Loss')
        ax.set_xlabel('Federated Rounds')
        ax.set_ylabel('Loss Value')
        ax.set_title('Federated Learning Training Progress')
        ax.legend()
        ax.grid(True, alpha=0.3)

    def _plot_communication_cost(self, ax):
        """繪製通訊成本分析"""
        rounds = self.training_results["training_history"]["rounds"]
        comm_cost = self.training_results["training_history"]["communication_cost"]

        ax.plot(rounds, np.array(comm_cost) / 1e6, 'g-', linewidth=2)
        ax.set_xlabel('Federated Rounds')
        ax.set_ylabel('Communication Cost (M parameters)')
        ax.set_title('Communication Cost Over Rounds')
        ax.grid(True, alpha=0.3)

        # 標注總通訊成本
        total_cost = sum(comm_cost) / 1e9
        ax.text(0.7, 0.9, f'Total: {total_cost:.2f}B params',
                transform=ax.transAxes, bbox=dict(boxstyle="round", facecolor='lightgreen'))

    def _plot_bs_efficiency_comparison(self, ax):
        """繪製各基站效率提升比較"""
        bs_stats = self.allocation_results["per_bs_stats"]
        bs_ids = list(bs_stats.keys())
        improvements = [stats["mean_improvement"] for stats in bs_stats.values()]
        std_errors = [stats["std_improvement"] for stats in bs_stats.values()]

        bars = ax.bar(bs_ids, improvements, yerr=std_errors,
                     color='skyblue', alpha=0.8, capsize=5)
        ax.set_xlabel('Base Station ID')
        ax.set_ylabel('Average Efficiency Improvement')
        ax.set_title('Per-Base Station Efficiency Gains')
        ax.grid(True, alpha=0.3)

        # 添加數值標籤
        for bar, improvement in zip(bars, improvements):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                   f'{improvement:.3f}', ha='center', va='bottom')

    def _plot_federated_vs_centralized(self, ax):
        """比較聯邦學習 vs 集中式學習"""
        # 模擬集中式學習結果（基於原始 Cell 4 結果）
        try:
            with open("training_summary_fixed.json", "r") as f:
                centralized_results = json.load(f)
            centralized_mae = centralized_results["results"]["Neural Network"]["MAE"]
        except:
            centralized_mae = 0.000832  # 來自原始結果

        federated_mae = self.training_results["final_metrics"]["test_mae"]

        methods = ['Centralized\nLearning', 'Federated\nLearning']
        mae_values = [centralized_mae, federated_mae]
        colors = ['orange', 'purple']

        bars = ax.bar(methods, mae_values, color=colors, alpha=0.8)
        ax.set_ylabel('Test MAE')
        ax.set_title('Centralized vs Federated Learning Performance')
        ax.grid(True, alpha=0.3)

        # 添加數值標籤
        for bar, mae in zip(bars, mae_values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{mae:.6f}', ha='center', va='bottom')

        # 標註隱私保護優勢
        ax.text(0.5, 0.8, '✓ Privacy Preserved\n✓ Distributed Training',
                transform=ax.transAxes, ha='center',
                bbox=dict(boxstyle="round", facecolor='lightblue', alpha=0.7))

    def _plot_allocation_patterns(self, ax):
        """分析資源分配模式"""
        # 提取分配模式數據
        allocations = []
        for result in self.allocation_results["federated_optimization_results"]:
            for bs_result in result["clients"].values():
                allocations.append(bs_result["optimized_allocation"])

        if allocations:
            allocations = np.array(allocations)
            slice_names = ['eMBB', 'URLLC', 'mMTC']

            # 繪製箱型圖
            ax.boxplot([allocations[:, i] for i in range(3)],
                      labels=slice_names, patch_artist=True,
                      boxprops=dict(facecolor='lightcoral', alpha=0.7))
            ax.set_ylabel('Allocated RBGs')
            ax.set_title('Resource Allocation Patterns Across Slices')
            ax.grid(True, alpha=0.3)
        else:
            ax.text(0.5, 0.5, 'No allocation data available',
                   transform=ax.transAxes, ha='center', va='center')

    def _plot_privacy_utility_tradeoff(self, ax):
        """繪製隱私-效用權衡"""
        # 模擬不同隱私參數下的效用
        noise_levels = [0.0, 0.1, 0.5, 1.0, 2.0]
        utility_scores = [1.0, 0.98, 0.92, 0.85, 0.75]  # 模擬數據
        privacy_scores = [0.0, 0.3, 0.6, 0.8, 0.95]     # 隱私保護程度

        # 當前配置點
        current_noise = self.training_results["config"]["dp_noise_multiplier"]
        current_utility = 0.98  # 基於當前結果
        current_privacy = 0.3

        ax.plot(privacy_scores, utility_scores, 'b-o', linewidth=2, markersize=6,
               label='Privacy-Utility Curve')
        ax.scatter(current_privacy, current_utility, color='red', s=100,
                  label=f'Current Config (noise={current_noise})', zorder=5)

        ax.set_xlabel('Privacy Protection Level')
        ax.set_ylabel('Model Utility Score')
        ax.set_title('Privacy-Utility Tradeoff Analysis')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # 標註最佳區域
        ax.axvspan(0.2, 0.4, alpha=0.2, color='green', label='Optimal Range')

    def generate_comparison_report(self):
        """生成聯邦學習 vs 集中式學習比較報告"""
        report = {
            "comparison_summary": {
                "federated_learning": {
                    "final_test_mae": self.training_results["final_metrics"]["test_mae"],
                    "training_rounds": len(self.training_results["training_history"]["rounds"]),
                    "total_training_time": self.training_results["total_training_time"],
                    "communication_cost": self.training_results["total_communication_cost"],
                    "privacy_preserved": True,
                    "distributed_training": True
                },
                "advantages": [
                    "Data privacy protection - raw data never leaves base stations",
                    "Distributed training reduces computational burden on central server",
                    "Better scalability for large-scale 5G networks",
                    "Robustness against single point of failure",
                    "Compliance with data protection regulations"
                ],
                "trade_offs": [
                    "Higher communication overhead",
                    "Longer training time due to coordination",
                    "Potential performance degradation due to privacy mechanisms",
                    "More complex implementation and debugging"
                ]
            },
            "efficiency_improvements": self.allocation_results["overall_stats"],
            "per_base_station_analysis": self.allocation_results["per_bs_stats"]
        }

        with open("federated_vs_centralized_comparison.json", "w") as f:
            json.dump(report, f, indent=2)

        print("📊 聯邦學習比較報告已生成")
        return report

# 執行視覺化
visualizer = FederatedResultsVisualizer(training_results, analysis_results)
visualizer.create_comprehensive_visualization()
comparison_report = visualizer.generate_comparison_report()

print("🎉 聯邦學習完整分析與視覺化完成!")
print("\n📋 生成檔案列表:")
print("   - federated_training_results.json")
print("   - federated_allocation_results.json")
print("   - federated_vs_centralized_comparison.json")
print("   - federated_comprehensive_results.png")
